In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm

def fishers_z_test(r1, r2, n1, n2):
    """
    Perform Fisher's Z-test to compare two correlation coefficients.
    r1, r2: Correlation coefficients to compare
    n1, n2: Sample sizes of the respective groups
    Returns: z-score and p-value
    """
    # Fisher's Z transformation
    z1 = 0.5 * np.log((1 + r1) / (1 - r1))
    z2 = 0.5 * np.log((1 + r2) / (1 - r2))

    # Standard error
    se = np.sqrt(1/(n1 - 3) + 1/(n2 - 3))

    # Z-test statistic
    z = (z1 - z2) / se

    # Two-tailed p-value
    p_value = 2 * (1 - norm.cdf(abs(z)))

    return z, p_value

n_human = 373
n_synthetic = 373


# --- Baseline (Humans) ---
# --- The cors take from the analysis/human-participants/human_participants_analysis.ipynb ---
correlations_human = {
    "Cabin Luggage": {"Extraversion": 0.10, "Agreeableness": 0.26, "Conscientiousness": 0.02, "Neuroticism": 0.03, "Openness": -0.03},
    "Packing Cubes": {"Extraversion": 0.17, "Agreeableness": 0.39, "Conscientiousness": -0.03, "Neuroticism": 0.00, "Openness": 0.04},
    "Water Bottle": {"Extraversion": 0.19, "Agreeableness": 0.26, "Conscientiousness": 0.05, "Neuroticism": -0.17, "Openness": -0.03}
}

# --- Model-specific correlations ---

# --- The cors take from the analysis/synthetic-participants/analysis_gpt_4o.ipynb ---
correlations_gpt4o = {
    "Cabin Luggage": {"Extraversion": 0.27, "Agreeableness": 0.21, "Conscientiousness": 0.26, "Neuroticism": 0.00, "Openness": 0.55},
    "Packing Cubes": {"Extraversion": 0.34, "Agreeableness": 0.23, "Conscientiousness": 0.23, "Neuroticism": 0.00, "Openness": 0.52},
    "Water Bottle": {"Extraversion": 0.64, "Agreeableness": 0.17, "Conscientiousness": 0.27, "Neuroticism": 0.08, "Openness": 0.57}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gpt_5_latest.ipynb ---
correlations_gpt5_latest = {
    "Cabin Luggage": {"Extraversion": 0.41, "Agreeableness": 0.31, "Conscientiousness": 0.30, "Neuroticism": 0.24, "Openness": 0.75},
    "Packing Cubes": {"Extraversion": 0.41, "Agreeableness": 0.30, "Conscientiousness": 0.27, "Neuroticism": 0.22, "Openness": 0.71},
    "Water Bottle": {"Extraversion": 0.72, "Agreeableness": 0.22, "Conscientiousness": -0.09, "Neuroticism": 0.24, "Openness": 0.61}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_1.5_flash.ipynb ---
correlations_gemini_1_5 = {
    "Cabin Luggage": {"Extraversion": 0.18, "Agreeableness": 0.24, "Conscientiousness": 0.28, "Neuroticism": 0.08, "Openness": 0.29},
    "Packing Cubes": {"Extraversion": 0.26, "Agreeableness": 0.22, "Conscientiousness": 0.19, "Neuroticism": 0.14, "Openness": 0.35},
    "Water Bottle": {"Extraversion": 0.37, "Agreeableness": 0.18, "Conscientiousness": 0.09, "Neuroticism": -0.21, "Openness": 0.42}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_2.0_flash.ipynb ---
correlations_gemini_2_0 = {
    "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_2.5_flash.ipynb ---
correlations_gemini_2_5 = {
    "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
}

fdr_threshold = 0.00022

traits = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']

# Function to compute Fisher's Z-tests
def fisher_tests(human_data, synthetic_data):
    results = {product: {} for product in human_data.keys()}
    
    for product in human_data.keys():
        for trait in human_data[product].keys():
            z, p = fishers_z_test(
                human_data[product][trait], 
                synthetic_data[product][trait], 
                n_human, 
                n_synthetic
            )
            results[product][trait] = {"z-score": z, "p-value": p}
    
    return results

results_gpt4o = fisher_tests(correlations_human, correlations_gpt4o)
results_gpt5_latest = fisher_tests(correlations_human, correlations_gpt5_latest)
results_gemini_1_5 = fisher_tests(correlations_human, correlations_gemini_1_5)
results_gemini_2_0 = fisher_tests(correlations_human, correlations_gemini_2_0)
results_gemini_2_5 = fisher_tests(correlations_human, correlations_gemini_2_5)



<h1>Assessing Alignment Between Humans and Models Using Fisher’s z-Test</h1>

In [37]:
import numpy as np
import scipy.stats as stats
import pandas as pd
from statsmodels.stats.multitest import multipletests

# --- The cors take from the analysis/synthetic-participants/analysis_gpt_4o.ipynb ---
correlations_gpt4o = {
    "Cabin Luggage": {"Extraversion": 0.27, "Agreeableness": 0.21, "Conscientiousness": 0.26, "Neuroticism": 0.00, "Openness": 0.55},
    "Packing Cubes": {"Extraversion": 0.34, "Agreeableness": 0.23, "Conscientiousness": 0.23, "Neuroticism": 0.00, "Openness": 0.52},
    "Water Bottle": {"Extraversion": 0.64, "Agreeableness": 0.17, "Conscientiousness": 0.27, "Neuroticism": 0.08, "Openness": 0.57}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gpt_5_latest.ipynb ---
correlations_gpt5_latest = {
    "Cabin Luggage": {"Extraversion": 0.41, "Agreeableness": 0.31, "Conscientiousness": 0.30, "Neuroticism": 0.24, "Openness": 0.75},
    "Packing Cubes": {"Extraversion": 0.41, "Agreeableness": 0.30, "Conscientiousness": 0.27, "Neuroticism": 0.22, "Openness": 0.71},
    "Water Bottle": {"Extraversion": 0.72, "Agreeableness": 0.22, "Conscientiousness": -0.09, "Neuroticism": 0.24, "Openness": 0.61}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_1.5_flash.ipynb ---
correlations_gemini_1_5 = {
    "Cabin Luggage": {"Extraversion": 0.18, "Agreeableness": 0.24, "Conscientiousness": 0.28, "Neuroticism": 0.08, "Openness": 0.29},
    "Packing Cubes": {"Extraversion": 0.26, "Agreeableness": 0.22, "Conscientiousness": 0.19, "Neuroticism": 0.14, "Openness": 0.35},
    "Water Bottle": {"Extraversion": 0.37, "Agreeableness": 0.18, "Conscientiousness": 0.09, "Neuroticism": -0.21, "Openness": 0.42}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_2.0_flash.ipynb ---
correlations_gemini_2_0 = {
    "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
}

# --- The cors take from the analysis/synthetic-participants/analysis_gemini_2.5_flash.ipynb ---
correlations_gemini_2_5 = {
    "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
    "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
}


df_human = pd.DataFrame(correlations_human).T
df_gpt4o = pd.DataFrame(correlations_gpt4o).T
df_gpt5_latest = pd.DataFrame(correlations_gpt5_latest).T
df_gemini_1_5 = pd.DataFrame(correlations_gemini_1_5).T
df_gemini_2_0 = pd.DataFrame(correlations_gemini_2_0).T
df_gemini_2_5 = pd.DataFrame(correlations_gemini_2_5).T


n_human = 373
n_synthetic = 373

def compute_fishers_z_per_product(df_human, df_model, model_name):
    results = []
    traits = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']

    for product in df_human.index:
        for trait in traits:
            r_human = df_human.loc[product, trait]
            r_model = df_model.loc[product, trait]
            z, p = fishers_z_test(r_human, r_model, n_human, n_synthetic)
            
            # Round values
            p = round(p, 4)
            z = round(z, 2)
            r_human = round(r_human, 2)
            r_model = round(r_model, 2)

            similarity_threshold = 0.1
            alignment = "Aligned" if (abs(r_human - r_model) <= similarity_threshold) and (p >= 0.05) else "Misaligned"
            significance = "*" if p < 0.00022 else "" # FDR-adjusted p-value

            results.append({
                "Product": product,
                "Trait-Matched Ad": trait,
                "Human ( r )": r_human,
                f"{model_name} ( r )": r_model,
                "Δr": r_human - r_model,
                "z-statistics": z,
                "p-value": p,
                "Significance": significance,
                "Alignment": alignment
            })

    df_results = pd.DataFrame(results)

    return df_results

df_z_gpt4o = compute_fishers_z_per_product(df_human, df_gpt4o, "GPT-4o")
df_z_gpt5_latest = compute_fishers_z_per_product(df_human, df_gpt5_latest, "GPT-5-latest")
df_z_gemini_1_5 = compute_fishers_z_per_product(df_human, df_gemini_1_5, "Gemini-1.5-flash")
df_z_gemini_2_0 = compute_fishers_z_per_product(df_human, df_gemini_2_0, "Gemini-2.0-flash")
df_z_gemini_2_5 = compute_fishers_z_per_product(df_human, df_gemini_2_5, "Gemini-2.5-flash")

In [32]:
df_z_gpt4o

,Product,Trait-Matched Ad,Human ( r ),GPT-4o ( r ),Δr,z-statistics,p-value,Significance,Alignment
0,Cabin Luggage,Openness,-0.03,0.55,-0.58,-8.82,0.0000,*,Misaligned
1,Cabin Luggage,Conscientiousness,0.02,0.26,-0.24,-3.41,0.0007,,Misaligned
2,Cabin Luggage,Extraversion,0.10,0.27,-0.17,-2.38,0.0173,,Misaligned
3,Cabin Luggage,Agreeableness,0.26,0.21,0.05,0.70,0.4851,,Aligned
4,Cabin Luggage,Neuroticism,0.03,-0.00,0.03,0.40,0.6902,,Aligned
5,Packing Cubes,Openness,0.04,0.52,-0.48,-7.24,0.0000,*,Misaligned
6,Packing Cubes,Conscientiousness,-0.03,0.23,-0.26,-3.58,0.0003,,Misaligned
7,Packing Cubes,Extraversion,0.17,0.34,-0.17,-2.46,0.0140,,Misaligned
8,Packing Cubes,Agreeableness,0.39,0.23,0.16,2.46,0.0140,,Misaligned
9,Packing Cubes,Neuroticism,-0.00,0.00,-0.00,-0.01,0.9902,,Aligned


In [ ]:
df_z_gpt5_latest

,Product,Trait-Matched Ad,Human ( r ),GPT-5-latest ( r ),Δr,z-statistics,p-value,Significance,Alignment
0,Cabin Luggage,Openness,-0.03,0.75,-0.78,-13.64,0.0000,*,Misaligned
1,Cabin Luggage,Conscientiousness,0.02,0.30,-0.28,-4.00,0.0001,*,Misaligned
2,Cabin Luggage,Extraversion,0.10,0.41,-0.31,-4.54,0.0000,*,Misaligned
3,Cabin Luggage,Agreeableness,0.26,0.31,-0.05,-0.76,0.4459,,Aligned
4,Cabin Luggage,Neuroticism,0.03,0.24,-0.21,-2.93,0.0034,,Misaligned
5,Packing Cubes,Openness,0.04,0.71,-0.67,-11.46,0.0000,*,Misaligned
6,Packing Cubes,Conscientiousness,-0.03,0.27,-0.30,-4.16,0.0000,*,Misaligned
7,Packing Cubes,Extraversion,0.17,0.41,-0.24,-3.57,0.0004,,Misaligned
8,Packing Cubes,Agreeableness,0.39,0.30,0.09,1.43,0.1519,,Aligned
9,Packing Cubes,Neuroticism,-0.00,0.22,-0.22,-3.05,0.0023,,Misaligned


In [39]:
df_z_gemini_1_5

,Product,Trait-Matched Ad,Human ( r ),Gemini-1.5-flash ( r ),Δr,z-statistics,p-value,Significance,Alignment
0,Cabin Luggage,Openness,-0.03,0.29,-0.32,-4.47,0.0000,*,Misaligned
1,Cabin Luggage,Conscientiousness,0.02,0.28,-0.26,-3.70,0.0002,*,Misaligned
2,Cabin Luggage,Extraversion,0.10,0.18,-0.08,-1.09,0.2758,,Aligned
3,Cabin Luggage,Agreeableness,0.26,0.24,0.02,0.27,0.7885,,Aligned
4,Cabin Luggage,Neuroticism,0.03,0.08,-0.05,-0.69,0.4891,,Aligned
5,Packing Cubes,Openness,0.04,0.35,-0.31,-4.37,0.0000,*,Misaligned
6,Packing Cubes,Conscientiousness,-0.03,0.19,-0.22,-3.01,0.0026,,Misaligned
7,Packing Cubes,Extraversion,0.17,0.26,-0.09,-1.26,0.2074,,Aligned
8,Packing Cubes,Agreeableness,0.39,0.22,0.17,2.60,0.0093,,Misaligned
9,Packing Cubes,Neuroticism,-0.00,0.14,-0.14,-1.93,0.0537,,Misaligned


In [40]:
df_z_gemini_2_0

,Product,Trait-Matched Ad,Human ( r ),Gemini-2.0-flash ( r ),Δr,z-statistics,p-value,Significance,Alignment
0,Cabin Luggage,Openness,-0.03,0.0,-0.03,-0.41,0.6812,,Aligned
1,Cabin Luggage,Conscientiousness,0.02,0.0,0.02,0.21,0.8309,,Aligned
2,Cabin Luggage,Extraversion,0.10,0.0,0.10,1.39,0.1660,,Aligned
3,Cabin Luggage,Agreeableness,0.26,0.0,0.26,3.60,0.0003,,Misaligned
4,Cabin Luggage,Neuroticism,0.03,0.0,0.03,0.40,0.6902,,Aligned
5,Packing Cubes,Openness,0.04,0.0,0.04,0.60,0.5465,,Aligned
6,Packing Cubes,Conscientiousness,-0.03,0.0,-0.03,-0.39,0.6952,,Aligned
7,Packing Cubes,Extraversion,0.17,0.0,0.17,2.36,0.0183,,Misaligned
8,Packing Cubes,Agreeableness,0.39,0.0,0.39,5.64,0.0000,*,Misaligned
9,Packing Cubes,Neuroticism,-0.00,0.0,-0.00,-0.01,0.9902,,Aligned


In [41]:
df_z_gemini_2_5

,Product,Trait-Matched Ad,Human ( r ),Gemini-2.5-flash ( r ),Δr,z-statistics,p-value,Significance,Alignment
0,Cabin Luggage,Openness,-0.03,0.0,-0.03,-0.41,0.6812,,Aligned
1,Cabin Luggage,Conscientiousness,0.02,0.0,0.02,0.21,0.8309,,Aligned
2,Cabin Luggage,Extraversion,0.10,0.0,0.10,1.39,0.1660,,Aligned
3,Cabin Luggage,Agreeableness,0.26,0.0,0.26,3.60,0.0003,,Misaligned
4,Cabin Luggage,Neuroticism,0.03,0.0,0.03,0.40,0.6902,,Aligned
5,Packing Cubes,Openness,0.04,0.0,0.04,0.60,0.5465,,Aligned
6,Packing Cubes,Conscientiousness,-0.03,0.0,-0.03,-0.39,0.6952,,Aligned
7,Packing Cubes,Extraversion,0.17,0.0,0.17,2.36,0.0183,,Misaligned
8,Packing Cubes,Agreeableness,0.39,0.0,0.39,5.64,0.0000,*,Misaligned
9,Packing Cubes,Neuroticism,-0.00,0.0,-0.00,-0.01,0.9902,,Aligned


In [21]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_alignment_with_humans(model="gpt-4o", similarity_threshold=0.1):
    # --- Baseline (Humans) ---
    correlations_human = {
        "Cabin Luggage": {"Extraversion": 0.10, "Agreeableness": 0.26, "Conscientiousness": 0.02, "Neuroticism": 0.03, "Openness": -0.03},
        "Packing Cubes": {"Extraversion": 0.17, "Agreeableness": 0.39, "Conscientiousness": -0.03, "Neuroticism": 0.00, "Openness": 0.04},
        "Water Bottle": {"Extraversion": 0.19, "Agreeableness": 0.26, "Conscientiousness": 0.05, "Neuroticism": -0.17, "Openness": -0.03}
    }

    # --- Model-specific correlations ---
    correlations_gpt4o = {
        "Cabin Luggage": {"Extraversion": 0.27, "Agreeableness": 0.21, "Conscientiousness": 0.26, "Neuroticism": 0.00, "Openness": 0.55},
        "Packing Cubes": {"Extraversion": 0.34, "Agreeableness": 0.23, "Conscientiousness": 0.23, "Neuroticism": 0.00, "Openness": 0.52},
        "Water Bottle": {"Extraversion": 0.64, "Agreeableness": 0.17, "Conscientiousness": 0.27, "Neuroticism": 0.08, "Openness": 0.57}
    }

    correlations_gpt5_latest = {
        "Cabin Luggage": {"Extraversion": 0.41, "Agreeableness": 0.31, "Conscientiousness": 0.30, "Neuroticism": 0.24, "Openness": 0.75},
        "Packing Cubes": {"Extraversion": 0.41, "Agreeableness": 0.30, "Conscientiousness": 0.27, "Neuroticism": 0.22, "Openness": 0.71},
        "Water Bottle": {"Extraversion": 0.72, "Agreeableness": 0.22, "Conscientiousness": -0.09, "Neuroticism": 0.24, "Openness": 0.61}
    }

    correlations_gemini_1_5 = {
        "Cabin Luggage": {"Extraversion": 0.18, "Agreeableness": 0.24, "Conscientiousness": 0.28, "Neuroticism": 0.08, "Openness": 0.29},
        "Packing Cubes": {"Extraversion": 0.26, "Agreeableness": 0.22, "Conscientiousness": 0.19, "Neuroticism": 0.14, "Openness": 0.35},
        "Water Bottle": {"Extraversion": 0.37, "Agreeableness": 0.18, "Conscientiousness": 0.09, "Neuroticism": -0.21, "Openness": 0.42}
    }

    correlations_gemini_2_0 = {
        "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
        "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
        "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
    }

    correlations_gemini_2_5 = {
        "Cabin Luggage": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
        "Packing Cubes": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00},
        "Water Bottle": {"Extraversion": 0.00, "Agreeableness": 0.00, "Conscientiousness": 0.00, "Neuroticism": 0.00, "Openness": 0.00}
    }

    MODELS = {
        "gpt-4o": correlations_gpt4o,
        "gpt-5-latest": correlations_gpt5_latest,
        "gemini-1.5-flash": correlations_gemini_1_5,
        "gemini-2.0-flash": correlations_gemini_2_0,
        "gemini-2.5-flash": correlations_gemini_2_5,
    }

    DISPLAY_NAME = {
        "gpt-4o": "GPT-4o",
        "gpt-5-latest": "GPT-5-latest",
        "gemini-1.5-flash": "Gemini 1.5 Flash",
        "gemini-2.0-flash": "Gemini 2.0 Flash",
        "gemini-2.5-flash": "Gemini 2.5 Flash",
    }

    if model not in MODELS:
        raise ValueError(f"Unknown model '{model}'. Valid options are: {list(MODELS.keys())}")

    model_name = DISPLAY_NAME[model]
    correlations_model = MODELS[model]

    # --- DataFrames ---
    df_human = pd.DataFrame(correlations_human).T
    df_model = pd.DataFrame(correlations_model).T

    # Keep trait order fixed
    traits = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']

    # Per-product x-axis ranges using just Human + selected model
    x_ranges = {}
    for product in df_human.index:
        all_vals = pd.concat([df_human.loc[product], df_model.loc[product]])
        x_ranges[product] = (float(all_vals.min()) - 0.05, float(all_vals.max()) + 0.05)

    # --- Figure ---
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=False,
        subplot_titles=list(df_human.index),
        vertical_spacing=0.1
    )

    # Draw per product
    for i, product in enumerate(df_human.index, start=1):
        for trait in traits:
            r_human = round(float(df_human.loc[product, trait]), 2)
            r_model = round(float(df_model.loc[product, trait]), 2)
            aligned = abs(r_human - r_model) <= similarity_threshold
            color = "green" if aligned else "red"

            # Connecting line
            fig.add_trace(go.Scatter(
                x=[r_human, r_model],
                y=[trait, trait],
                mode='lines',
                line=dict(color="black", width=1),
                opacity=0.7,
                showlegend=False
            ), row=i, col=1)

            # Human marker
            fig.add_trace(go.Scatter(
                x=[r_human],
                y=[trait],
                mode='markers+text',
                marker=dict(size=10, symbol="circle", color=color),
                text=[f"r={r_human}"],
                textposition="middle left" if r_human < r_model else "middle right",
                showlegend=False
            ), row=i, col=1)

            # Model marker
            fig.add_trace(go.Scatter(
                x=[r_model],
                y=[trait],
                mode='markers+text',
                marker=dict(size=10, symbol="square", color=color),
                text=[f"r={r_model}"],
                textposition="middle right" if r_human < r_model else "middle left",
                showlegend=False
            ), row=i, col=1)

        # Axis formatting
        fig.update_yaxes(
            categoryorder='array',
            categoryarray=traits[::-1],
            row=i, col=1
        )
        fig.update_xaxes(
            title_text="Correlation",
            range=x_ranges[product],
            row=i, col=1
        )

    # Legends
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, symbol='circle', color='gray'),
        name='Humans'
    ))
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, symbol='square', color='gray'),
        name='Synthetic Twin'
    ))
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, symbol='circle', color='green'),
        name='Aligned'
    ))
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, symbol='circle', color='red'),
        name='Misaligned'
    ))

    fig.update_layout(
        height=1400,
        width=1300,
        title=f"Alignment Between Humans and Synthetic Twins ({model_name})",
        margin=dict(t=100, b=80, l=100, r=220),
        legend=dict(
            x=1.02,
            y=1,
            traceorder='normal',
            font=dict(size=12),
            borderwidth=0
        )
    )

    fig.show()


In [22]:
plot_alignment_with_humans("gpt-4o")

In [23]:
plot_alignment_with_humans("gpt-5-latest")

In [24]:
plot_alignment_with_humans("gemini-1.5-flash")

In [25]:
plot_alignment_with_humans("gemini-2.0-flash")

In [2]:
import plotly.graph_objects as go

PRODUCTS = ["P1 — Cabin Luggage", "P2 — Packing Cubes", "P3 — Water Bottle"]

# GPT
gpt4o_same_cat = [60.0, 80.0, 80.0]
gpt5_latest_same_cat = [60.0, 80.0, 80.0]

# Gemini
gemini_1_5_flash_same_cat = [80.0, 80.0, 60.0]
gemini_2_0_flash_same_cat = [60.0, 80.0, 80.0]
gemini_2_5_flash_same_cat = [00.0, 00.0, 00.0]

human_baseline_top = 100.0

fig = go.Figure()

# GPT-4o bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gpt4o_same_cat,
    name="GPT-4o",
    text=[f"{v:.0f}%" for v in gpt4o_same_cat],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#4C72B0"),
    opacity=0.8,
    textfont=dict(size=12)
))

# GPT-5-latest bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gpt5_latest_same_cat,
    name="GPT-5-latest",
    text=[f"{v:.0f}%" for v in gpt5_latest_same_cat],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#1F77B4"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Gemini 1.5 Flash bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gemini_1_5_flash_same_cat,
    name="Gemini-1.5 Flash",
    text=[f"{v:.0f}%" for v in gemini_1_5_flash_same_cat],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#D17ACD"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Gemini 2.0 Flashbars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gemini_2_0_flash_same_cat,
    name="Gemini-2.0 Flash",
    text=[f"{v:.0f}%" for v in gemini_2_0_flash_same_cat],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#A64CA6"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Keep outside texts from being clipped
fig.update_traces(cliponaxis=False)

# Human baseline (100% perfect agreement)
fig.add_hline(
    y=human_baseline_top, 
    line_dash="solid", 
    line_color="black",
    annotation_text="Human baseline (perfect agreement) = 100%",
    annotation_position="top left",
)

# Layout
fig.update_layout(
    xaxis_title="Product",
    yaxis_title="% Same Category",
    yaxis_range=[0, 105],
    bargap=0.35,
    barmode="group",
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(showline=True, linewidth=1, linecolor="black", mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor="black", mirror=True, gridcolor="lightgrey"),
    height=550,
    width=950,
    # More breathing room for top labels and bottom footnote
    margin=dict(l=70, r=40, t=90, b=130),
    legend=dict(orientation="h", yanchor="bottom", y=1.04, xanchor="left", x=0)
)

# Footnote — move slightly up and rely on extra bottom margin
fig.add_annotation(
    text=(
        "Note: Bars show the proportion of identical rating categories between humans and synthetic twins. Categories are derived by mean ratings on a 5-point<br>"
        "Likert scale. For example, Human Mean = 3.45 is Category 3 and Synthetic Twins Mean (GPT-4o) = 3.40 is Category 3 (same category, agreement);<br>"
        "whereas Human Mean = 3.45 is Category 3 and Synthetic Twins Mean (Gemini-1.5 Flash) = 3.60 is Category 4 (different categories, no agreement).<br>"
        "For more details, see Table X - Table Y of detailed agreement analysis."
    ),

    xref="paper", yref="paper",
    x=0.48, y=-0.20,
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    xanchor="center",
    yanchor="top"
)

fig


In [7]:
import plotly.graph_objects as go

PRODUCTS = ["P1 — Cabin Luggage", "P2 — Packing Cubes", "P3 — Water Bottle"]

# GPT
gpt4o_r            = [0.87, 0.88, 0.83]
gpt5_latest_r      = [0.86, 0.91, 0.87]

# Gemini
gemini_1_5_flash_r = [0.81, 0.84, 0.81]
gemini_2_0_flash_r = [0.88, 0.88, 0.87]

human_baseline_top = 1.0  # perfect agreement

fig = go.Figure()

# GPT-4o bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gpt4o_r,
    name="GPT-4o",
    text=[f"r={v:.2f}" for v in gpt4o_r],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#4C72B0"),
    opacity=0.8,
    textfont=dict(size=12)
))

# GPT-5-latest bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gpt5_latest_r,
    name="GPT-5-latest",
    text=[f"r={v:.2f}" for v in gpt5_latest_r],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#1F77B4"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Gemini 1.5 Flash bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gemini_1_5_flash_r,
    name="Gemini-1.5 Flash",
    text=[f"r={v:.2f}" for v in gemini_1_5_flash_r],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#D17ACD"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Gemini 2.0 Flash bars
fig.add_trace(go.Bar(
    x=PRODUCTS,
    y=gemini_2_0_flash_r,
    name="Gemini-2.0 Flash",
    text=[f"r={v:.2f}" for v in gemini_2_0_flash_r],
    textposition="outside",
    marker=dict(line=dict(color="black", width=1.5), color="#A64CA6"),
    opacity=0.8,
    textfont=dict(size=12)
))

# Keep outside texts from being clipped
fig.update_traces(cliponaxis=False)

# Human (perfect) baseline
fig.add_hline(
    y=human_baseline_top,
    line_dash="solid",
    line_color="black",
    annotation_text="Human baseline (perfect agreement) = 1.00",
    annotation_position="top left",
)

# Layout
fig.update_layout(
    xaxis_title="Product",
    yaxis_title="Aggregate correlations",
    yaxis_range=[0, 1.05],
    bargap=0.35,
    barmode="group",
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(showline=True, linewidth=1, linecolor="black", mirror=True),
    yaxis=dict(showline=True, linewidth=1, linecolor="black", mirror=True, gridcolor="lightgrey"),
    height=550,
    width=950,
    margin=dict(l=70, r=40, t=90, b=130),
    legend=dict(orientation="h", yanchor="bottom", y=1.04, xanchor="left", x=0)
)

# Footnote — tailored for correlations
fig.add_annotation(
    text=(
        "Note: Bars show Pearson correlations between human and synthetic twins’ mean ratings across the five personality traits <br>"
        "for each product. A correlation of r = 1.00 indicates perfect alignment with human ratings (i.e., identical relative patterns across traits),<br>"
        "whereas r = 0 indicates no alignment. For more details, see Table X - Table Y of detailed agreement analysis."
    ),
    xref="paper", yref="paper",
    x=0.48, y=-0.20,
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    xanchor="center",
    yanchor="top"
)

fig.show()
